## Properties of materials

This example shows how to:
   - use / modify predefined materials
   - setup refraction index and light dispersion in glass-like material
   - add textures materials
   
Tkinter GUI window is launched from the notebook in this example. This allows re-running code cells and see results without scrolling back to the figure.

![notebook output image](https://plotoptix.rnd.team/images/refractions_dispersion_textures.jpg "This notebook output")

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.materials import m_clear_glass, m_diffuse # predefined materials
from plotoptix.utils import make_color_2d
from plotoptix.enums import RtFormat # texture buffer formats

Make some data first:

In [2]:
rx = (-20, 20)
rz = (-20, 20)
n = 160

x = np.linspace(rx[0], rx[1], n)
z = np.linspace(rz[0], rz[1], n)

X, Z = np.meshgrid(x, z)

# positions of blocks
data = np.stack((X.flatten(), np.zeros(n*n), Z.flatten())).T
# heights of blocks
v = np.zeros(data.shape); v[:,1] = 0.1 + 0.05 * np.random.rand(n*n)
# XZ sizes
size_u = 0.98 * (rx[1] - rx[0]) / (n - 1)
size_w = 0.98 * (rz[1] - rz[0]) / (n - 1)

# sphere center and radius:
sc = [0.8, 1, 0.3]
sr = 0.85

m = 300
# radii of particles:
r = 0.03 * np.random.rand(m) + 0.01
# positions of particles:
p = sc + 2*sr * (np.random.rand(m,3) - 0.5)

Setup the raytracer using Tkinter GUI as the output target:

In [3]:
optix = TkOptiX()
optix.set_param(min_accumulation_step=4,     # set more accumulation frames to get rid of the noise
                max_accumulation_frames=512,
                light_shading="Hard")        # use "Hard" shading for best caustics
optix.set_uint("path_seg_range", 12, 32)     # more path segments to allow multiple refractions

Only *diffuse* material is available by default. Other materials need to be configured before using. Let's make some more material definitions starting from predefined values:

In [4]:
m_textured = m_diffuse.copy()
m_air = m_clear_glass.copy()

optix.setup_material("tex", m_textured)
optix.setup_material("glass", m_clear_glass)
optix.setup_material("air", m_air)

Add objects to the scene.

In [5]:
optix.set_data("blocks", pos=data,
               u=[size_u, 0, 0], v=v, w=[0, 0, size_w],
               c = np.random.rand(n*n),
               geom="Parallelepipeds")

optix.set_data("bigblock", geom="Parallelepipeds", mat="tex",
               pos=[-0.8, 0.15, -0.8], u=[0.7,0,0], v=[0,0.7,0], w=[0,0,0.7], c=0.94)
optix.rotate_primitive("bigblock", 0, rot=(0, np.pi/8, 0))

optix.set_data("sphere", pos=sc, r=sr, c=10, mat="glass")

optix.set_data("particles", pos=p, r=r, c=10, mat="air")

Setup a good point of view, set background and lights. Use ligth shading best for caustics.

In [6]:
optix.setup_camera("cam1", cam_type="DoF",
                   eye=[0.45, 1.27, -3], target=[0.48, 0.6, 0.44], up=[-0.003, 0.94, -0.33],
                   aperture_radius=0.025, fov=55, focal_scale=0.61)

optix.setup_light("light1", pos=[4, 5.1, 3], color=[12, 11, 10], radius=1.9)
optix.setup_light("light2", pos=[-1.5, 3, -2], color=[8, 9, 10], radius=0.2)
optix.set_background(0)
optix.set_ambient(0)

exposure = 0.4; gamma = 2.2 
optix.set_float("tonemap_exposure", exposure)
optix.set_float("tonemap_igamma", 1 / gamma)
optix.add_postproc("Gamma")  # apply gamma correction postprocessing stage, or
#optix.setup_denoiser()      # use AI denoiser (exposure and gamma are applied as well)

Open the GUI.

In [7]:
optix.start()

Display *glass* parameters, note constant refraction index for all RGB components:

In [8]:
print(m_clear_glass)

{'ClosestHitPrograms': ['0::path_tracing_materials.ptx::glass_closest_hit'], 'AnyHitPrograms': ['1::path_tracing_materials.ptx::any_hit'], 'VarInt': {'material_flags': 12}, 'VarFloat': {'radiation_length': 0.0, 'vol_scattering': 1.0, 'light_emission': 0.0}, 'VarFloat3': {'refraction_index': [1.4, 1.4, 1.4], 'surface_albedo': [1.0, 1.0, 1.0]}}


Change refraction index, causing light dispersion. Wait for a few frames until green bias disappears:

In [9]:
m_clear_glass["VarFloat3"]["refraction_index"] = [1.38, 1.4, 1.43]
optix.update_material("glass", m_clear_glass, refresh=True)

m_air["VarFloat3"]["refraction_index"] = [1.38, 1.4, 1.43] # same as in glass, so particles nested in the sphere are not visible
optix.update_material("air", m_clear_glass, refresh=True)

Change refraction index so particles in the void dissapear and those in the sphere become air bubbles:

In [10]:
m_air["VarFloat3"]["refraction_index"] = [1, 1, 1]
optix.update_material("air", m_clear_glass, refresh=True)

Let's see how the *diffuse* material is defined:

In [11]:
print(m_textured)

{'ClosestHitPrograms': ['0::path_tracing_materials.ptx::diffuse_closest_hit'], 'AnyHitPrograms': ['1::path_tracing_materials.ptx::any_hit'], 'VarInt': {'material_flags': 2}}


Add texture from the file. Note that format has to be ``Float4`` (this is the texture format, source file can be anything: color or grayscale). Use ``Gamma`` field to preserve texture image colors in postprocessing.

In [12]:
m_textured["Textures"]=[
        {
          "Source": r"data/side_d.png",
          "Format": RtFormat.Float4.value,
          "Gamma": gamma
        }
      ]
optix.update_material("tex", m_textured, refresh=True)

Mutiple textures are projected on different walls of geometry primitives.

In [13]:
m_textured["Textures"]=[
        {
          "Source": r"data/side_c.png",
          "Format": RtFormat.Float4.value,
          "Gamma": gamma  
        },
        {
          "Source": r"data/side_b.png",
          "Format": RtFormat.Float4.value,
          "Gamma": gamma
        },
        {
          "Source": r"data/side_d.png",
          "Format": RtFormat.Float4.value,
          "Gamma": gamma
        }
      ]
optix.update_material("tex", m_textured, refresh=True)

Texture can be prepared algorithmically. In this case ``Width`` and ``Height`` have to be specified, and the texture has to be passed as a flat list of RGBA values; ``Format`` is like previously: ``Float4``.

In [14]:
mx = (-1, 1)
mz = (-1, 1)
nm = 20

x = np.linspace(mx[0], mx[1], nm)
z = np.linspace(mz[0], mz[1], nm)

Mx, Mz = np.meshgrid(x, z)
M = np.abs(Mx) ** 4 + np.abs(Mz) ** 4
M = 1 - (0.5 / np.max(M)) * M

M = make_color_2d(M, gamma=gamma) # make RGB array, accounting for gamma correction

Ma = np.zeros((nm, nm, 4))        # convert to RGBA format
Ma[...,:-1] = M

In [15]:
m_diffuse["Textures"]=[
        {
          "Width": nm,
          "Height": nm,
          "DataArray": Ma.flatten().tolist(),
          "Format": RtFormat.Float4.value
        }
      ]
optix.update_material("diffuse", m_diffuse, refresh=True)

Close GUI window, release resources.

In [16]:
optix.close()